## Chapter 6 - 게이트가 추가된 RNN


#### 6.1 RNN의 문제점
---
앞 장에서 설명한 RNN은 시계열 데이터의 장기 의존 관계를 학습하기 어렵습니다. 

그 이유는 BPTT에서 기울기 소실 혹은 기울기 폭발이 일어나기 때문입니다.

![](../image/image_63.png)

h<sub>t</sub> = tanh(h<sub>t-1</sub>W<sub>h</sub> + x<sub>t</sub>W<sub>x</sub>+b)

임을 잘 기억하자.

![](../image/image_64.png)

![](../image/image_65.png)

![](../image/image_66.png)

![](../image/image_67.png)

##### 1. 파이토치에서의 기울기 클리핑
파이토치에서는 그레디언트 클리핑으로 불린다(사실 기울기가 그레디언트....)
> 파이토치 : https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html#torch.nn.utils.clip_grad_norm_

> 참고 글 : https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-6/05-gradient-clipping

간단히 구현해보자

```py
def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate
```
이때 grads는 기울기의 리스트 max_norm은 문턱값을 뜻한다.

파이토치에서도 비슷하게 구현이 되어 있다.

1. torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2.0, error_if_nonfinite=False)
    - parameters (Iterable[Tensor] or Tensor) : 기울기가 정규화될 텐서 또는 단일 텐서의 반복 가능
    - max_norm : 문턱값
    - norm_type : type of the used p-norm. Can be 'inf' for infinity norm.
    - error_if_nonfinite : 참인 경우 매개 변수의 그라디언트의 전체 표준이 nan, inf 또는 -inf인 경우 오류가 발생합니다. 기본값: false

</br>

2. 사용법

    ```py
    import torch.optim as optim
    import torch.nn.utils as torch_utils

    learning_rate = 1.
    max_grad_norm = 5.

    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    # In orther to avoid gradient exploding, we apply gradient clipping.
    torch_utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    # Take a step of gradient descent.
    optimizer.step()
    ```

#### 6.2 기울기 소실과 LSTM
---

기울기 폭발에 대한 해결책은 기울기 클리핑이여도 여전히 기울기 소실에 대해서는 해결을 하지 못했다.

RNN의 기울기 소실에 대한 문제를 해결하기 위해서 '게이트'를 추가했는데 이 것이 바로 '게이트가 추가된 RNN'이다. 대표적으로 LSTM과 GRU가 있다. 

![](../image/image_68.png)

위에서 보듯 LSTM계층의 인터페이스에는 c라는 경로가 있다는 차이가 있습니다. 이 c를 기억 셀(혹은 단순히 셀)이라 하며 LSTM 전용의 기억 메커니즘이랍니다.

기억 셀의 특징은 데이터를 자기 자신으로만(LSTM 계층 내에서만) 주고받는다는 것입니다. 즉, LSTM 계층 내에서만 완결되고, 다른 계층으로는 출력하지 않습니다. 

반면LSTM의 은닉상태 h는 RNN 계층과 마찬가지로 다른 계층으로(위쪽으로) 출력됩니다.


![](../image/image_69.png)

![](../image/image_70.png)

![](../image/image_71.png)

![](../image/image_72.png)

![](../image/image_73.png)

![](../image/image_74.png)

##### 1. 파이토치의 LSTM
 

> https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM

> https://justkode.kr/deep-learning/pytorch-rnn

![](../image/image_75.png)

![](../image/image_76.png)

사용예제

```py
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
```

#### 6.3 RNNLM 추가 개선
---

![](../image/image_77.png)

##### 1. 드롭아웃에 의한 과적합 억제
![](../image/image_77.png)

![](../image/image_78.png)

![](../image/image_79.png)

![](../image/image_81.png)

위 그림에서 보듯 같은 계층의 드롭아웃끼리 마스크를 공유함으로써 마스크가 ‘고정’됩니다. 

그 결과 정보 를 잃게 되는 방법도 ‘고정’되므로, 일반적인 드롭아웃 때와 달리 정부가 지수적으로 손실되는 사태를 피할 수 있습니다

![](../image/image_82.png)